Load Data and combine tracking data

In [ ]:
import pandas as pd
import statistics as stats
import os
games = pd.read_csv('../input/nfl-big-data-bowl-2021/games.csv')
players = pd.read_csv('../input/nfl-big-data-bowl-2021/players.csv')
plays = pd.read_csv('../input/nfl-big-data-bowl-2021/plays.csv')
week1 = pd.read_csv('../input/nfl-big-data-bowl-2021/week1.csv')
week2 = pd.read_csv('../input/nfl-big-data-bowl-2021/week2.csv')
week3 = pd.read_csv('../input/nfl-big-data-bowl-2021/week3.csv')
week4 = pd.read_csv('../input/nfl-big-data-bowl-2021/week4.csv')
week5 = pd.read_csv('../input/nfl-big-data-bowl-2021/week5.csv')
week6 = pd.read_csv('../input/nfl-big-data-bowl-2021/week6.csv')
week7 = pd.read_csv('../input/nfl-big-data-bowl-2021/week7.csv')
week8 = pd.read_csv('../input/nfl-big-data-bowl-2021/week8.csv')
week9 = pd.read_csv('../input/nfl-big-data-bowl-2021/week9.csv')
week10 = pd.read_csv('../input/nfl-big-data-bowl-2021/week10.csv')
week11 = pd.read_csv('../input/nfl-big-data-bowl-2021/week11.csv')
week12 = pd.read_csv('../input/nfl-big-data-bowl-2021/week12.csv')
week13 = pd.read_csv('../input/nfl-big-data-bowl-2021/week13.csv')
week14 = pd.read_csv('../input/nfl-big-data-bowl-2021/week14.csv')
week15 = pd.read_csv('../input/nfl-big-data-bowl-2021/week15.csv')
week16 = pd.read_csv('../input/nfl-big-data-bowl-2021/week16.csv')
week17 = pd.read_csv('../input/nfl-big-data-bowl-2021/week17.csv')
coverages_week1 = pd.read_csv('../input/nfl-big-data-bowl-2021-bonus/coverages_week1.csv')
targeted_receiver = pd.read_csv('../input/nfl-big-data-bowl-2021-bonus/targetedReceiver.csv')
frames = [week1, week2, week3, week4, week5, week6, week7, week8, week9, week10, week11, week12, week13, week14, week15, week16, week17]
tracking_original = pd.concat(frames, ignore_index = True)
tracking_original

Reorient tracking data to all face the same way

In [ ]:
tracking_df = tracking_original.copy()
tracking_df.loc[tracking_df['playDirection'] == 'left', 'x'] = 120 - tracking_df['x']
tracking_df.loc[tracking_df['playDirection'] == 'left', 'y'] = 53.33 - tracking_df['y']
tracking_df.loc[(tracking_df['playDirection'] == 'left') & (tracking_original['o'] <= 180), 'o'] = tracking_df['o'] + 180
tracking_df.loc[(tracking_df['playDirection'] == 'left') & (tracking_original['o'] > 180), 'o'] = tracking_df['o'] - 180
tracking_df.loc[(tracking_df['playDirection'] == 'left') & (tracking_original['dir'] <= 180), 'dir'] = tracking_df['dir'] + 180
tracking_df.loc[(tracking_df['playDirection'] == 'left') & (tracking_original['dir'] > 180), 'dir'] = tracking_df['dir'] - 180

add week 1 coverages and targeted receiver to each play

In [ ]:
plays = pd.merge(plays, coverages_week1, how='left')
plays = pd.merge(plays, targeted_receiver, how='left')
plays = pd.merge(plays, players, how='left', left_on='targetNflId', right_on='nflId')
plays.iloc[0]

start plays expanded, our eventual training data

In [ ]:
plays_expanded = plays[['gameId', 'playId', 'down', 'yardsToGo', 'penaltyCodes', 'penaltyJerseyNumbers', 'passResult', 'offensePlayResult', 'playResult', 'epa', 'nflId', 'displayName']]
plays_expanded

limit tracking data to frame where ball is released

In [ ]:
def tracking_filter(week_df):
    release = week_df['event']=='pass_forward'
    release_df = week_df[release]
    return release_df
release_df = tracking_filter(tracking_df)
release_df

merge receiver tracking with plays expanded

In [ ]:
plays_expanded = pd.merge(plays_expanded, release_df, how='left', on = ['gameId', 'playId', 'nflId'], suffixes=('', '_y'))
plays_expanded.drop(plays_expanded.filter(regex='_y$').columns.tolist(),axis=1, inplace=True)
plays_expanded.drop(columns = ['time', 'dis', 'event'], inplace = True)
plays_expanded.rename(columns = {'team': 'team_o', 'nflId': 'nflId_o', 'displayName': 'displayName_o', 'x': 'x_o', 'y': 'y_o', 's': 'speed_o', 'a': 'acceleration_o', 'o': 'orientation_o', 'dir': 'dir_o', 'jerseyNumber': 'jerseyNumber_o', 'position': 'position_o'}, inplace = True)
plays_expanded.columns

merge football tracking with plays expanded

In [ ]:
def football_filter(week_df):
    football_df = week_df[week_df.displayName.isin(['Football'])]
    return football_df
football_df = football_filter(release_df)
plays_expanded = pd.merge(plays_expanded, football_df, how='left', on = ['gameId', 'playId'], suffixes=('', '_y'))
plays_expanded.drop(plays_expanded.filter(regex='_y$').columns.tolist(),axis=1, inplace=True)
plays_expanded.drop(columns = ['team', 'time', 'dis', 'event', 'o', 'dir', 'nflId', 'jerseyNumber', 'position', 'displayName'], inplace = True)
plays_expanded.rename(columns = {'x': 'x_b', 'y': 'y_b', 's': 'speed_b', 'a': 'acceleration_b'}, inplace = True)
plays_expanded.iloc[0]

returns dataframe of defenders and distance to targeted receiver at time of release

In [ ]:
temp_df = []
import math
def closest_defender_finder():
    def_pos = ['DL', 'FS', 'DB', 'DE', 'OLB', 'SS', 'CB', 'ILB', 'S', 'MLB', 'NT', 'LB']
    temp_df = pd.merge(plays_expanded, release_df, how='right', on = ['gameId', 'playId'], suffixes=('', '_y'))
    temp_df.drop(plays_expanded.filter(regex='_y$').columns.tolist(),axis=1, inplace=True)
    temp_df = temp_df[temp_df.position.isin(def_pos)]
    temp_df['distance_from_receiver'] = (((temp_df['x']-temp_df['x_o'])**2)+((temp_df['y']-temp_df['y_o'])**2))**(1/2)
    new_df = pd.DataFrame() 
    new_df['closest_defender_id'] = temp_df['nflId']
    new_df['defender_name'] = temp_df['displayName']
    new_df['closest_defender_dist'] = temp_df['distance_from_receiver']
    new_df['gameId'] = temp_df['gameId']
    new_df['playId'] = temp_df['playId']
    new_df['frameId'] = temp_df['frameId']
    return temp_df, new_df
    
temp_df, new_df = closest_defender_finder()
new_df

In [ ]:
display(new_df.sort_values(['gameId', 'playId', 'frameId', 'closest_defender_dist']))
closest_defenders_release_df = new_df.sort_values(['gameId', 'playId', 'frameId', 'closest_defender_dist']).drop_duplicates(subset=['gameId', 'playId', 'frameId'], keep='first')
closest_defenders_release_df = closest_defenders_release_df.reset_index(drop=True)
display(closest_defenders_release_df)

returns dataframe of closest defender on every target at time of release

In [ ]:
# distance_dict = {}
# new_df = new_df.reset_index(drop = True)
# for i in range(0, len(new_df)):
#     if new_df.loc[i, 'id'] not in distance_dict:
#         distance_dict[new_df.loc[i, 'id']] = [[new_df.loc[i, 'defender_id']], [new_df.loc[i, 'distance_from_receiver']]]
#     else:
#         distance_dict[new_df.loc[i, 'id']][0].append(new_df.loc[i, 'defender_id'])
#         distance_dict[new_df.loc[i, 'id']][1].append(new_df.loc[i, 'distance_from_receiver'])
# final_df = pd.DataFrame()
# index = 0
# for x in distance_dict:
#     m = min(distance_dict[x][1])
#     i = distance_dict[x][1].index(min(distance_dict[x][1]))
#     final_df.loc[index, 'id'] = x
#     final_df.loc[index, 'closest_defender_id'] = distance_dict[x][0][i]
#     final_df.loc[index, 'defender_dist'] = m
#     index += 1
# final_df

split id column back into game, play, and frame

In [ ]:
# id_df = final_df['id'].str.split(' ', expand = True)
# final_df['gameId'] = id_df[0].astype(int)
# final_df['playId'] = id_df[1].astype(int)
# final_df['frameId'] = id_df[2].astype(float)
# final_df.drop(columns = ['id'], inplace = True)
# final_df

merge closest defender to plays expanded

In [ ]:
plays_expanded = pd.merge(plays_expanded, closest_defenders_release_df, how='left', on = ['gameId', 'playId', 'frameId'])
plays_expanded = pd.merge(plays_expanded, release_df, how='left', right_on = ['gameId', 'playId', 'nflId'], left_on = ['gameId', 'playId', 'closest_defender_id'], suffixes=('', '_y'))
plays_expanded.drop(plays_expanded.filter(regex='_y$').columns.tolist(),axis=1, inplace=True)
plays_expanded.drop(columns = ['time', 'dis', 'event', 'nflId'], inplace = True)
plays_expanded.rename(columns = {'team': 'team_d', 'displayName': 'displayName_d', 'x': 'x_d', 'y': 'y_d', 's': 'speed_d', 'a': 'acceleration_d', 'o': 'orientation_d', 'dir': 'dir_d', 'jerseyNumber': 'jerseyNumber_d', 'position': 'position_d'}, inplace = True)
plays_expanded.iloc[0]

In [ ]:
print('hello george')

In [ ]:
import numpy as np
def display_info(df, game_id, play_id, frame_id):
    frame_indices = np.where((df['gameId'] == game_id) & (df['playId'] == play_id) & (df['frameId'] == frame_id))
    display(df.loc[frame_indices])

display_info(tracking_df,2018102110,1710, 1)

In [ ]:
def filter_tracking_frames_by_event(df, event_types):
    frame_indices = df['event'].isin(event_types)
    return df[frame_indices]

def drop_duplicate_arrive_incomplete(df):
#     display(df.sort_values('event'))
#     display(df.sort_values(['nflId', 'gameId','playId']))
#     display(df[df.duplicated(['gameId', 'playId', 'nflId'], keep=False)].sort_values(['nflId', 'gameId','playId']))
    fixed_df = df.sort_values(['nflId', 'gameId','playId','event']).drop_duplicates(subset=['gameId', 'playId', 'nflId'], keep='first')
#     display(fixed_df.sort_values('event'))
    return fixed_df.sort_values('event')

# DELETE later
#     display(df.groupby(['playId', 'nflId']).apply(lambda x: x.loc[x.event == 'pass_arrived']))
#     indices = df.groupby(['playId', 'nflId']).event.transform(lambda x: x)
#     pd.set_option('display.max_rows', len(indices))
#     print(indices)
#     pd.reset_option('display.max_rows')
#     df = df.loc[df.event == indices]

def drop_fumble_plays(tracking_df, fixed_df):
    tracking_df = filter_tracking_frames_by_event(tracking_df, ['fumble_defense_recovered'])
    display(tracking_df.sort_values(['playId']))
    play_ids = pd.Series(tracking_df['playId']).unique()
    display(play_ids)
    frame_indices = ~fixed_df['playId'].isin(play_ids)
    return fixed_df[frame_indices]
    

# Filter frames for pass_arrived events
pass_arrived_df = filter_tracking_frames_by_event(tracking_df, ['pass_arrived', 'pass_outcome_incomplete'])
# pd.set_option('display.max_rows', len(pass_arrived_df['event']))
# display(pd.Series(pass_arrived_df['event']))
# pd.reset_option('display.max_rows')
fixed_df = drop_duplicate_arrive_incomplete(pass_arrived_df)
fixed_df.loc[fixed_df.event == 'pass_outcome_incomplete', 'event'] = 'pass_arrived'
display(fixed_df)

# display(set(pd.Series(tracking_df['event']).unique()))

fixed_df = drop_fumble_plays(tracking_df, fixed_df)
display(fixed_df)

In [ ]:
def find_closest_defenders_on_arrival(plays_expanded, df):
    defenders_on_arrival_df = []
    
    def_pos = ['DL', 'FS', 'DB', 'DE', 'OLB', 'SS', 'CB', 'ILB', 'S', 'MLB', 'NT', 'LB']
    
#     display(plays_expanded)
#     display(df)
    
    defenders_on_arrival_df = pd.merge(plays_expanded, df, how='right', on = ['gameId', 'playId'], suffixes=('', '_y'))
#     display(defenders_on_arrival_df)
    
    defenders_on_arrival_df.drop(plays_expanded.filter(regex='_y$').columns.tolist(),axis=1, inplace=True)
    defenders_on_arrival_df = defenders_on_arrival_df[defenders_on_arrival_df.position.isin(def_pos)]
    
#     display(defenders_on_arrival_df)
    
    defenders_on_arrival_df['distance_from_receiver'] = (((defenders_on_arrival_df['x'] - defenders_on_arrival_df['x_o']) ** 2)
                                                         + ((defenders_on_arrival_df['y'] - defenders_on_arrival_df['y_o']) ** 2)
                                                         ) ** (1/2)
    
    closest_defenders_arrival_df = pd.DataFrame()
    closest_defenders_arrival_df['closest_defender_id'] = defenders_on_arrival_df['nflId']
    closest_defenders_arrival_df['defender_name'] = defenders_on_arrival_df['displayName']
    closest_defenders_arrival_df['closest_defender_dist'] = defenders_on_arrival_df['distance_from_receiver']
    closest_defenders_arrival_df['gameId'] = defenders_on_arrival_df['gameId']
    closest_defenders_arrival_df['playId'] = defenders_on_arrival_df['playId']
    closest_defenders_arrival_df['frameId'] = defenders_on_arrival_df['frameId']
    return defenders_on_arrival_df, closest_defenders_arrival_df

defenders_on_arrival_df, closest_defenders_arrival_df = find_closest_defenders_on_arrival(plays_expanded, fixed_df)
display(defenders_on_arrival_df)
display(closest_defenders_arrival_df)

In [ ]:
sorted_closest_defenders_arrival_df = closest_defenders_arrival_df.sort_values(['gameId', 'playId', 'frameId', 'closest_defender_dist'])
display(sorted_closest_defenders_arrival_df)

# Drop closest defender which was found at release
display(new_df.sort_values(['gameId', 'playId', 'frameId', 'closest_defender_dist']))

sorted_closest_defenders_release_df = closest_defenders_release_df.sort_values(['gameId', 'playId', 'frameId', 'closest_defender_dist'])
display(sorted_closest_defenders_release_df)

# display(sorted_closest_defenders_arrival_df.append(sorted_closest_defenders_release_df))
sorted_closest_defenders_arrival_exclusion_df = sorted_closest_defenders_arrival_df.append(sorted_closest_defenders_release_df).drop_duplicates(subset=['gameId', 'playId', 'frameId', 'closest_defender_id'],keep=False)
display(sorted_closest_defenders_arrival_exclusion_df)

grouped_df = sorted_closest_defenders_arrival_exclusion_df.groupby(['gameId', 'playId', 'frameId']) #as_index=False

# count = 0
# for key, item in grouped_df:
#     print(key)
#     print(item)
#     if count > 5: break
#     else: count += 1
top_three_defenders_arrival_exclusion = grouped_df.head(3)
display(top_three_defenders_arrival_exclusion)

# def find_top_three_defenders():

# top_three_defenders_exclusion = sorted_closest_defenders_arrival_exclusion_df.groupby(['gameId', 'playId', 'frameId']).closest_defender_dist.transform(lambda x: x )
# sorted_closest_defenders_arrival_exclusion_df = sorted_closest_defenders_arrival_exclusion_df.loc[sorted_closest_defenders_arrival_exclusion_df.closest_defender_dist == top_three_defenders_exclusion]

In [ ]:
# Finds closest defender at ball arrival
# closest_defenders_arrival_df = closest_defenders_arrival_df.sort_values(['gameId', 'playId', 'frameId', 'closest_defender_dist']).drop_duplicates(subset=['gameId', 'playId', 'frameId'], keep='first')
# closest_defenders_arrival_df = closest_defenders_arrival_df.reset_index(drop=True)
# display(closest_defenders_arrival_df)

In [ ]:
# distance_dict = {}
# new_df = new_df.reset_index(drop = True)
# for i in range(0, len(new_df)):
#     if new_df.loc[i, 'id'] not in distance_dict:
#         distance_dict[new_df.loc[i, 'id']] = [[new_df.loc[i, 'defender_id']], [new_df.loc[i, 'distance_from_receiver']]]
#     else:
#         distance_dict[new_df.loc[i, 'id']][0].append(new_df.loc[i, 'defender_id'])
#         distance_dict[new_df.loc[i, 'id']][1].append(new_df.loc[i, 'distance_from_receiver'])
# final_df = pd.DataFrame()
# index = 0
# for x in distance_dict:
#     m = min(distance_dict[x][1])
#     i = distance_dict[x][1].index(min(distance_dict[x][1]))
#     final_df.loc[index, 'id'] = x
#     final_df.loc[index, 'closest_defender_id'] = distance_dict[x][0][i]
#     final_df.loc[index, 'defender_dist'] = m
#     index += 1
# final_df

In [ ]:
# id_df = final_df['id'].str.split(' ', expand = True)
# final_df['gameId'] = id_df[0].astype(int)
# final_df['playId'] = id_df[1].astype(int)
# final_df['frameId'] = id_df[2].astype(float)
# final_df.drop(columns = ['id'], inplace = True)
# final_df

# Adding receiver and ball at arrival

In [ ]:
def arrival_filter(week_df):
    arrival = week_df['event']=='pass_arrived'
    arrival_df = week_df[arrival]
    return arrival_df
arrival_df = arrival_filter(tracking_df)
plays_expanded = pd.merge(plays_expanded, arrival_df, how='left', left_on = ['gameId', 'playId', 'nflId_o'], right_on = ['gameId', 'playId', 'nflId'],suffixes=('', '_y'))
plays_expanded.drop(plays_expanded.filter(regex='_y$').columns.tolist(),axis=1, inplace=True)
plays_expanded.drop(columns = ['time', 'dis', 'event', 'team', 'displayName', 'jerseyNumber', 'position', 'nflId'], inplace = True)
plays_expanded.rename(columns = {'x': 'x_o_a', 'y': 'y_o_a', 's': 'speed_o_a', 'a': 'acceleration_o_a', 'o': 'orientation_o_a', 'dir': 'dir_o_a'}, inplace = True)
plays_expanded.iloc[0]

In [ ]:
def football_arrival_filter(week_df):
    football_arrival_df = week_df[week_df.displayName.isin(['Football'])]
    return football_arrival_df
football_arrival_df = football_filter(arrival_df)
plays_expanded = pd.merge(plays_expanded, football_arrival_df, how='left', on = ['gameId', 'playId'], suffixes=('', '_y'))
plays_expanded.drop(plays_expanded.filter(regex='_y$').columns.tolist(),axis=1, inplace=True)
plays_expanded.drop(columns = ['team', 'time', 'dis', 'event', 'o', 'dir', 'nflId', 'jerseyNumber', 'position', 'displayName'], inplace = True)
plays_expanded.rename(columns = {'x': 'x_b_a', 'y': 'y_b_a', 's': 'speed_b_a', 'a': 'acceleration_b_a'}, inplace = True)
plays_expanded.loc[0]